In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm, notebook
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/crypto-news-mar-2023/articles-crypto.csv


#### Code for fetching data from news api.

In [2]:
# !python -m pip install newsapi-python

# from newsapi import NewsApiClient

# API_KEY = '856737b219bf4835acc6352ea966c67b'
# newsapi = NewsApiClient(api_key=API_KEY)

# from datetime import datetime, timedelta

# def get_past_time_range(days=30):
#     past_days = {}
#     for day in range(1, days):
#         from_day = str(datetime.now().date() - timedelta(days=day))
#         to_day = str(datetime.now().date() - timedelta(days=day - 1))
#         past_days.update({from_day:to_day})
#     return past_days

# get_past_time_range(30)

# def get_articles(query, days=30):
#     past_days_filter = get_past_time_range(days)
#     all_articles = []
#     for fromDate,toDate in notebook.tqdm(past_days_filter.items()):
#         for page in range(1, 3):
#             try:
#                 page_articles = newsapi.get_everything(q=query, language='en', from_param=fromDate, to=toDate,sort_by='relevancy', page=page)['articles']
#                 if len(page_articles) == 0: break
#                 all_articles.extend(page_articles)
#             except:
#                 break
            
#     return all_articles

# articles = get_articles('cryptocurrency')

# len(articles)

# articles[0]

# articles_df = pd.DataFrame(articles)
# articles_df.head()

# articles_df.to_csv('articles-crypto.csv')

In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8c8891432e0aec2b6e8b097b9ea4677e0d272b540c4106a3a0a6e3f6864b39b5
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
articles_df = pd.read_csv('/kaggle/input/crypto-news-mar-2023/articles-crypto.csv')
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16T05:00:03Z,On 28 October 2022 Elon Musk tweeted the bird ...
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16T11:34:00Z,Silicon Valley had $74.9 billion in venture-ca...
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16T15:00:17Z,"If you own an Android smartphone, then the pla..."
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16T13:30:17Z,Crypto assets have existed for over a decade b...
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16T10:12:06Z,"OKB, the utility token of popular cryptocurren..."


In [7]:
titles = articles_df['title'].values.tolist()
titles[:5]

['Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 'Remote Working Increases VC Investments in Other Areas Besides Silicon Valley',
 'Despite the best efforts of Google Play Protect, the Play Store is not as safe as it may appear',
 '5 Myths About Crypto Regulations Debunked',
 "OKB Unfazed By BTC's Retreat, Gains 50% In Weekly Charts"]

In [8]:
embeddings = model.encode(titles)
embeddings[:5]

Batches:   0%|          | 0/177 [00:00<?, ?it/s]

array([[ 0.02549198,  0.08546034,  0.08514886, ..., -0.08972151,
         0.02734277, -0.00249079],
       [ 0.10810093, -0.08295441,  0.06143335, ..., -0.09770083,
        -0.03455804, -0.02146049],
       [-0.08645321,  0.02147196,  0.01853899, ..., -0.07560316,
         0.12599476,  0.13990094],
       [-0.0264598 ,  0.01820142, -0.04395117, ..., -0.03163949,
         0.05570024,  0.02574591],
       [ 0.00991424, -0.00632995,  0.07007281, ..., -0.12255175,
        -0.06314813, -0.00612311]], dtype=float32)

In [9]:
len(embeddings), len(embeddings[0])

(5653, 384)

Reference link - https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e

In [10]:
!pip install umap-learn
!pip install hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=3427913 sha256=1df316a63f1775cf865c117ce8c

In [11]:
import umap
import hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      random_state = None):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
    """
    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors, 
                                n_components=n_components, 
                                metric='cosine', 
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [13]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given cluster supplied from running hdbscan
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [14]:
def random_search(embeddings, space, num_evals):
    """
    Randomly search hyperparameter space and limited number of times 
    and return a summary of the results
    """
    
    results = []
    
    for i in notebook.trange(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        n_components = random.choice(space['n_components'])
        min_cluster_size = random.choice(space['min_cluster_size'])
        
        clusters = generate_clusters(embeddings, 
                                     n_neighbors = n_neighbors, 
                                     n_components = n_components, 
                                     min_cluster_size = min_cluster_size, 
                                     random_state = 97)
    
        label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
                
        results.append([i, n_neighbors, n_components, min_cluster_size, 
                        label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'n_components', 
                                               'min_cluster_size', 'label_count', 'cost'])
    
    return result_df.sort_values(by='cost')

#### Tuning parameters

In [15]:
# space = {
#     "n_neighbors": range(12, 16),
#     "n_components": range(3, 7),
#     "min_cluster_size": range(2, 16),
#     "random_state": 97
# }
# results = random_search(embeddings, space, 100)

In [16]:
#results

In [17]:
clusters = generate_clusters(embeddings, 15, 6, 15, 97)
len(clusters.labels_), len(embeddings)

(5653, 5653)

In [18]:
clusters.labels_[:5]

array([27, 61, 82, 93, -1])

In [19]:
articles_df['clusterLabel'] = clusters.labels_
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16T05:00:03Z,On 28 October 2022 Elon Musk tweeted the bird ...,27
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16T11:34:00Z,Silicon Valley had $74.9 billion in venture-ca...,61
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16T15:00:17Z,"If you own an Android smartphone, then the pla...",82
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16T13:30:17Z,Crypto assets have existed for over a decade b...,93
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16T10:12:06Z,"OKB, the utility token of popular cryptocurren...",-1


In [20]:
cluster_27 = articles_df['title'][articles_df['clusterLabel'] == 27][:10].values.tolist()
cluster_27

['Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 '234 Million Shiba Inu (SHIB) Tokens Burned as Network Maintains High Load',
 'Shiba Inu Spikes On 60% Spurt In Volume',
 'Litecoin Network Reaches ATH in Hashrate, Boosting Security and Stability',
 'Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 'Koyo Token Holders Surpasses 1,170 Amid Increased Shiba Inu Burns',
 '234 Million Shiba Inu (SHIB) Tokens Burned as Network Maintains High Load',
 'Lead Shiba Inu Developer Stresses More $SHIB Burns Are Needed to ‘Move Price’',
 'Shibarium’s Puppynet Testnet: Massive Interest with 5.8M Wallets and 3.2M+ Transactions',
 "Dogecoin Creator Finally 'Reveals' How DOGE Is Related to Bitcoin"]

In [21]:
articles_df['publishedAt'] = pd.to_datetime(articles_df['publishedAt'])
articles_df['date'] = articles_df['publishedAt'].dt.date
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel,date
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16 05:00:03+00:00,On 28 October 2022 Elon Musk tweeted the bird ...,27,2023-04-16
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16 11:34:00+00:00,Silicon Valley had $74.9 billion in venture-ca...,61,2023-04-16
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16 15:00:17+00:00,"If you own an Android smartphone, then the pla...",82,2023-04-16
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16 13:30:17+00:00,Crypto assets have existed for over a decade b...,93,2023-04-16
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16 10:12:06+00:00,"OKB, the utility token of popular cryptocurren...",-1,2023-04-16


#### Count of dates for news on cluster label - 27 (for eg)

In [24]:
dates = articles_df[articles_df['clusterLabel'] == 27]['date'].values.tolist()
len(set(dates))

29

In [25]:
clusters_12_6_5 = generate_clusters(embeddings, 12, 6, 5, 97)
articles_df['clusterLabel_12_6_5'] = clusters_12_6_5.labels_

In [26]:
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel,date,clusterLabel_12_6_5
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16 05:00:03+00:00,On 28 October 2022 Elon Musk tweeted the bird ...,27,2023-04-16,258
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16 11:34:00+00:00,Silicon Valley had $74.9 billion in venture-ca...,61,2023-04-16,41
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16 15:00:17+00:00,"If you own an Android smartphone, then the pla...",82,2023-04-16,301
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16 13:30:17+00:00,Crypto assets have existed for over a decade b...,93,2023-04-16,308
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16 10:12:06+00:00,"OKB, the utility token of popular cryptocurren...",-1,2023-04-16,363


In [27]:
cluster_27 = articles_df['title'][articles_df['clusterLabel_12_6_5'] == 27][:10].values.tolist()
cluster_27

['Bob Lee, Cash App founder and former Square executive, dead at age 43',
 'Cash App Creator Bob Lee Passes Away After Incident',
 'Cash App founder Bob Lee dies at 43',
 'Bob Lee, Cash App founder and former Square executive, dead at age 43',
 'Cash App Creator Bob Lee Passes Away After Incident',
 'Cash App founder Bob Lee dies at 43']

In [29]:
articles_df['clusterProb'] = clusters.probabilities_
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel,date,clusterLabel_12_6_5,clusterProb
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16 05:00:03+00:00,On 28 October 2022 Elon Musk tweeted the bird ...,27,2023-04-16,258,1.000000
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16 11:34:00+00:00,Silicon Valley had $74.9 billion in venture-ca...,61,2023-04-16,41,0.750751
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16 15:00:17+00:00,"If you own an Android smartphone, then the pla...",82,2023-04-16,301,0.649419
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16 13:30:17+00:00,Crypto assets have existed for over a decade b...,93,2023-04-16,308,1.000000
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16 10:12:06+00:00,"OKB, the utility token of popular cryptocurren...",-1,2023-04-16,363,0.000000


In [31]:
clusters_df = articles_df.copy()
clusters_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel,date,clusterLabel_12_6_5,clusterProb
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16 05:00:03+00:00,On 28 October 2022 Elon Musk tweeted the bird ...,27,2023-04-16,258,1.000000
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16 11:34:00+00:00,Silicon Valley had $74.9 billion in venture-ca...,61,2023-04-16,41,0.750751
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16 15:00:17+00:00,"If you own an Android smartphone, then the pla...",82,2023-04-16,301,0.649419
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16 13:30:17+00:00,Crypto assets have existed for over a decade b...,93,2023-04-16,308,1.000000
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16 10:12:06+00:00,"OKB, the utility token of popular cryptocurren...",-1,2023-04-16,363,0.000000


In [32]:
clusters_df.drop(clusters_df[clusters_df.clusterLabel == -1].index, inplace=True)
len(clusters_df)

4310

In [43]:
unique_events = clusters_df.sort_values('clusterProb', ascending = False).groupby(['clusterLabel', 'date'], as_index = False).first()
unique_events.head()

,clusterLabel,date,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel_12_6_5,clusterProb
0,0,2023-03-27,4072,"{'id': None, 'name': 'Yahoo Entertainment'}",AMANCAI BIRABEN,South Dakota gov's veto of cryptocurrency regu...,Proponents had argued the bill would have cent...,https://finance.yahoo.com/news/south-dakota-go...,https://s.yimg.com/ny/api/res/1.2/.rrRtZMlsVwJ...,2023-03-27 19:08:59+00:00,South Dakota's House failed Monday to override...,43,1.000000
1,1,2023-03-20,5258,"{'id': None, 'name': 'CoinDesk'}",Will Canny,Cryptocurrency Market Set for More Upside Afte...,The current environment is the perfect setting...,https://www.coindesk.com/markets/2023/03/20/cr...,https://www.coindesk.com/resizer/Jt5EkyPkJ2Fbv...,2023-03-20 12:15:04+00:00,Financial markets have been shaken in the past...,3,1.000000
2,1,2023-03-22,5211,"{'id': None, 'name': 'Biztoc.com'}",azcoinnews.com,Charles Hoskinson Calls Out Credit Suisse's Di...,"In a podcast on March 20, Cardano founder Char...",https://biztoc.com/x/563ee938d888d7ad,https://c.biztoc.com/p/563ee938d888d7ad/og.webp,2023-03-22 11:08:09+00:00,"In a podcast on March 20, Cardano founder Char...",336,0.999628
3,1,2023-03-25,4472,"{'id': None, 'name': 'Biztoc.com'}",apnews.com,"Venezuela: 21 officials, businessmen arrested ...",Venezuela’s attorney general said Saturday tha...,https://biztoc.com/x/8c2cd893530eacdd,https://c.biztoc.com/p/8c2cd893530eacdd/og.webp,2023-03-25 20:26:06+00:00,Venezuelas attorney general said Saturday that...,0,0.932199
4,1,2023-03-27,4153,"{'id': None, 'name': 'International Business T...",Nica Osorio,"Venezuela Nabs 21 Officials, Businessmen Linke...",The government released arrest warrants for 11...,https://www.ibtimes.com/venezuela-nabs-21-offi...,https://d.ibtimes.com/en/full/4434873/venezuel...,2023-03-27 05:27:03+00:00,KEY POINTS\r\n<ul><li>The attorney general's o...,0,0.933630


In [44]:
len(unique_events)

1166

In [45]:
unique_events.groupby(['clusterLabel']).count()

,date,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel_12_6_5,clusterProb
clusterLabel,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1
1,7,7,7,7,7,7,7,7,7,7,7,7
2,4,4,4,4,4,4,4,4,4,4,4,4
3,7,7,7,7,7,7,7,7,7,7,7,7
4,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
107,13,13,13,13,13,13,13,13,13,13,13,13
108,13,13,13,13,13,13,13,13,13,13,13,13
109,13,13,13,13,13,13,13,13,13,13,13,13


In [46]:
unique_events[['title', 'date']][unique_events['clusterLabel']==27]

,title,date
181,Dogecoin (DOGE) Price Prediction 2025-2030: DO...,2023-03-19
182,Shiba Inu Sees Outflow Of $270 Million From Cr...,2023-03-20
183,MopsSwap: The 1st DEX on The Shibarium Ecosystem!,2023-03-21
184,Daily Briefing: Latest on Amritpal Singh crack...,2023-03-22
185,"A brief history of Elon Musk's pets, including...",2023-03-23
186,5 Alternative Meme Coins To Dogecoin (DOGE),2023-03-24
187,Dogecoin Has Unveiled A Massive Update,2023-03-25
188,Shiba Inu: Funny Money or a Compelling Crypto ...,2023-03-26
189,Shiba Inu Consolidates Despite Rising Burn Rate,2023-03-27
190,Bitcoin and Shiba Inu Are Beating the Nasdaq 1...,2023-03-28


In [47]:
unique_events[['title', 'date']][unique_events['clusterLabel']==51]

,title,date
406,Venezuela overhauls national crypto department,2023-03-19
407,MetaMask enables direct crypto purchases in Ni...,2023-03-21
408,MetaMask Partners With MoonPay To Enable Insta...,2023-03-22
409,Exchanges and Mining Farms Shut Down in Venezuela,2023-03-24
410,"Venezuela shuts down crypto mining facilities,...",2023-03-25
411,Venezuela Shuts Down Various Crypto Exchanges ...,2023-03-26
412,Bitcoin platform with over one million Nigeria...,2023-04-05
413,Success of Genesis Market Takedown Attempt Cal...,2023-04-06
414,MetaMask launches new fiat purchase function f...,2023-04-10
415,MetaMask introduces a new fiat purchase functi...,2023-04-11


In [69]:
unique_events[['title', 'date']][unique_events['clusterLabel']==51]

,title,date
406,Venezuela overhauls national crypto department,2023-03-19
407,MetaMask enables direct crypto purchases in Ni...,2023-03-21
408,MetaMask Partners With MoonPay To Enable Insta...,2023-03-22
409,Exchanges and Mining Farms Shut Down in Venezuela,2023-03-24
410,"Venezuela shuts down crypto mining facilities,...",2023-03-25
411,Venezuela Shuts Down Various Crypto Exchanges ...,2023-03-26
412,Bitcoin platform with over one million Nigeria...,2023-04-05
413,Success of Genesis Market Takedown Attempt Cal...,2023-04-06
414,MetaMask launches new fiat purchase function f...,2023-04-10
415,MetaMask introduces a new fiat purchase functi...,2023-04-11


In [71]:
unique_events[['title', 'date']][unique_events['clusterLabel']==111].values.tolist()

[['“Black Swan” Author Ridicules $1 Million Bitcoin Price Prediction',
  datetime.date(2023, 3, 19)],
 ['Here is What 2023 Holds for Bitcoin, According to BitMEX',
  datetime.date(2023, 3, 20)],
 ['Bitcoin halving one year away; When will BTC price spike?',
  datetime.date(2023, 3, 21)],
 ['Bitcoin at $100,000? Insiders say the cryptocurrency could test new highs this year',
  datetime.date(2023, 3, 23)],
 ['Bitcoin, Other Cryptos See $360 Million Sold Off In Single Day – Here’s Why',
  datetime.date(2023, 3, 24)],
 ['Bitcoin Maximalist Matthew Kratter Asserts $1 Million Value For BTC, Counters Balaji’s Forecast',
  datetime.date(2023, 3, 25)],
 ['Jim Cramer’s bitcoin prediction flops as coin posts 14% gain. Are people actually still listening to this guy?',
  datetime.date(2023, 3, 27)],
 ['Is Bitcoin Going to $100,000 -- or $0?', datetime.date(2023, 3, 31)],
 ['Bitcoin (BTC) Price Prediction 2025-2030: Will BTC succumb to bearish pressure',
  datetime.date(2023, 4, 2)],
 ['Calm in th

In [72]:
unique_events[['title', 'date']][unique_events['clusterLabel']==105].values.tolist()

[['How to Store Your Crypto Safely: A Guide to Cryptocurrency Wallets 2023',
  datetime.date(2023, 3, 20)],
 ['What Kim Kardashian, Brad Pitt, And Cheryl Burke Can Teach Us About Dividing Unconventional Assets Throughout A Divorce',
  datetime.date(2023, 3, 22)],
 ['Secure Your Crypto: Safeguarding Tips for Investment',
  datetime.date(2023, 3, 24)],
 ['Traditional finance fears drive digital asset investment inflows to $160M',
  datetime.date(2023, 3, 27)],
 ['Crypto Investors Urged To Use Cold Wallets To Avoid Loss',
  datetime.date(2023, 3, 28)],
 ['Experts from Amberdata, Custodia Bank, and United States for Kraken discuss the role digital assets are going to play in the future of finance',
  datetime.date(2023, 3, 29)],
 ['Powerbridge Technologies Acquires Majority Stake in DTI Group to Build Trade Digitalization Ecosystem',
  datetime.date(2023, 3, 30)],
 ['The People Have Spoken And They Want Real Assets',
  datetime.date(2023, 3, 31)],
 ['Crypto Funding: Ledger and EigenLabs Le